# 支持向量机 Support Vector Machines

本节包括以下内容：
1. 间隔 Margin
2. 记号 Notation
3. 函数间隔和几何间隔 Functional and geometric margins
4. 最大间隔分类器 The optimal margin classifier
5. 拉格朗日对偶性 Lagrange duality
6. 最大间隔分类器 Optimal margin classifier
7. 核方法 Kernels
8. 正则化、不可分的情况 Regularization and the non-separable case
9. SMO算法 The SMO algorithm
10. SMO
